In [2]:
!pip install trax


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os 
import shutil
import random as rnd

# import relevant libraries
import trax
import trax.fastmath.numpy as np
from trax import layers as tl
from trax import fastmath

In [21]:
import string
import re
import os
import nltk
nltk.download('twitter_samples')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples 

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import pandas as pd

### Loading Tweets

In [32]:
df = pd.read_csv(r'covid_tweets_preprocessed-Copy1.csv')
print(df)

           tweet_id  manual score  machine score  sentiment_score  \
0      1.240000e+18             0              1         0.116071   
1      1.240000e+18            -1             -1        -0.050000   
2      1.240000e+18            -1             -1        -0.714286   
3      1.240000e+18             0              1         0.700000   
4      1.240000e+18             0              0         0.000000   
...             ...           ...            ...              ...   
11452  1.270000e+18             0              0         0.000000   
11453  1.270000e+18            -1             -1        -0.071429   
11454  1.270000e+18            -1             -1        -0.200000   
11455  1.270000e+18            -1             -1        -0.243056   
11456  1.270000e+18            -1             -1        -0.092593   

                                                    text  \
0      Wuhan has been in complete quarantine for over...   
1      RT @RedTRaccoon: You can no longer defend th

In [33]:
df['text'] = df['text'].astype(str)

In [34]:
df.shape

(11457, 9)

In [35]:
# Split the data into pos/neu/neg, sampling at the minimum length == 2689

df_positive = df[df['machine score'] == 1].sample(2689)
# df_neutral = df[df['machine score'] == 0].sample(2689)
df_negative = df[df['machine score'] == -1].sample(2689)

In [36]:
train_positive = df_positive.sample(frac=0.8,random_state=200)
test_positive = df_positive.drop(train_positive.index)

In [37]:
# train_neutral = df_neutral.sample(frac=0.8,random_state=200)
# test_neutral = df_neutral.drop(train_neutral.index)

In [38]:
train_negative = df_negative.sample(frac=0.8,random_state=200)
test_negative = df_negative.drop(train_negative.index)

In [39]:
import os 
import shutil
import random as rnd

# import relevant libraries
import trax
import trax.fastmath.numpy as np
from trax import layers as tl
from trax import fastmath

In [40]:
train = pd.concat([train_positive, train_negative], axis = 0)
test = pd.concat([test_positive, test_negative], axis = 0)

# train = pd.concat([train_positive, train_negative], axis = 0)
# test = pd.concat([test_positive, test_negative], axis = 0)

In [41]:
test.head()

,tweet_id,manual score,machine score,sentiment_score,text,date,language,favorite_count,month
2808,1.240000e+18,0,1,0.125000,RT @TamangPhan: Why are people getting judged ...,Thu Mar 19 20:01:16 +0000 2020,en,0,Mar
6216,1.260000e+18,1,1,0.150000,RT @kpnorcal: A 70 year old COVID-19 patient i...,Tue May 19 04:23:49 +0000 2020,en,0,May
1942,1.240000e+18,0,1,0.437500,RT @You_grow_wai: Hello Fam üíô Ghana Has Con...,Thu Mar 19 19:58:34 +0000 2020,en,0,Mar
7878,1.260000e+18,1,1,0.150000,RT @pythoroshan: @sudhirchaudhary @ZeeNews Abs...,Tue May 19 04:28:59 +0000 2020,en,0,May
3666,1.250000e+18,0,1,0.318182,"RT @jerry_jtaylor: Per @dana1981, COVID-denial...",Sat Apr 18 04:51:06 +0000 2020,en,0,Apr


In [42]:
def load_tweets():
    # Load positive and negative tweets
    df = pd.read_csv(r'covid_tweets_preprocessed-Copy1.csv')
    df['text'] = df['text'].astype(str)

    df_positive = df[df['manual score'] == 1].sample(2689)
#     df_neutral = df[df['manual score'] == 0].sample(2689)
    df_negative = df[df['manual score'] == -1].sample(2689)
    
    df_train_positive = df_positive.sample(frac=0.8,random_state=200)
    df_test_positive = df_positive.drop(df_train_positive.index)
    
#     df_train_neutral = df_neutral.sample(frac=0.8,random_state=200)
#     df_test_neutral = df_neutral.drop(df_train_neutral.index)
    
    df_train_negative = df_negative.sample(frac=0.8,random_state=200)
    df_test_negative = df_negative.drop(df_train_negative.index)
    
    train_pos = list(df_train_positive['text'])
    test_pos = list(df_test_positive['text'])
    
#     train_neu = list(df_train_neutral['text'])
#     test_neu = list(df_test_neutral['text'])   
    
    train_neg = list(df_train_negative['text'])
    test_neg = list(df_test_negative['text'])
    
    # Combine training data into one set
#     train_x = train_pos + train_neu + train_neg 
    train_x = train_pos + train_neg 

    # Combine testidation data into one set
#     test_x  = test_pos + test_neu + test_neg
    test_x  = test_pos + test_neg

    # Set the labels for the training set (1 for positive, 0 for neutral, -1 for negative)
#     train_y = np.append(np.append(np.ones(len(train_pos)), np.zeros(len(train_neu))), -1*np.ones(len(train_neg)))
    train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
    
    # Set the labels for the test set (1 for positive, 0 for negative, -1 for negative)
#     train_y = np.append(np.append(np.ones(len(test_pos)), np.zeros(len(test_neu))), -1*np.ones(len(test_neg)))
    test_y  = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))


    return train_pos, train_neg, train_x, train_y, test_pos, test_neg, test_x, test_y

In [43]:
train_pos, train_neg, train_x, train_y, test_pos, test_neg, test_x, test_y = load_tweets()

In [44]:
train_y

DeviceArray([1., 1., 1., ..., 0., 0., 0.], dtype=float32)

### Processing Tweets

In [102]:
def process_tweet(tweet):
    '''
    Input: 
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet
    
    '''
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https.*?\s', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    ### START CODE HERE ###
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and # remove stopwords
            word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
    ### END CODE HERE ###
    return tweets_clean

In [105]:
tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

In [106]:
stopwords_english = stopwords.words('english')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

### Get Vocabualary

In [107]:
process_tweet(train_x[5])

['remind', 'anyon', 'certain', 'pm', 'name', 'scumo']

In [109]:
type(train_x)
ls = {type(item) for item in train_x}
print(ls)

{<class 'str'>}


In [110]:
def get_vocab(train_x):

    # Include special tokens 
    # started with pad, end of line and unk tokens
    Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2} 

    # Note that we build vocab using training data
    for tweet in train_x: 
        processed_tweet = process_tweet(tweet)
        for word in processed_tweet:
            if word not in Vocab: 
                Vocab[word] = len(Vocab)
    
    return Vocab

Vocab = get_vocab(train_x)

In [111]:
len(Vocab)

8823

### Convert words to vector

In [112]:
def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):
    '''
    Input: 
        tweet - A string containing a tweet
        vocab_dict - The words dictionary
        unk_token - The special string for unknown tokens
        verbose - Print info durign runtime
    Output:
        tensor_l - A python list with
        
    '''     
    word_l = process_tweet(tweet)
    
    if verbose:
        print("List of words from the processed tweet:")
        print(word_l)
        
    tensor_l = [] 
    
    unk_ID = vocab_dict[unk_token]
    
    if verbose:
        print(f"The unique integer ID for the unk_token is {unk_ID}")
        
    for word in word_l:

        word_ID = vocab_dict[word] if word in vocab_dict else unk_ID
        tensor_l.append(word_ID)
    
    return tensor_l

In [113]:
print("Actual tweet is\n", test_x[0])
print("\nTensor of tweet:\n", tweet_to_tensor(test_x[0], vocab_dict=Vocab))

Actual tweet is
 RT @MaryanneChisho2: Hi everyone, just an update - My daughters COVID-19 test was negative. We are relieved!

Tensor of tweet:
 [1946, 1403, 1720, 40, 31, 81, 13, 4359, 2]


### Create a Batch Generator

In [98]:
# def data_generator(data_pos, data_neu, data_neg, batch_size, loop, vocab_dict, shuffle=False):
#     '''
#     Input: 
#         data_pos - Set of positive examples
#         data_neu - Set of neutral examples
#         data_neg - Set of negative examples
#         batch_size - number of samples per batch. Must be a multiple of three
#         loop - True or False
#         vocab_dict - The words dictionary
#         shuffle - Shuffle the data order
#     Yield:
#         inputs - Subset of positive and negative examples
#         targets - The corresponding labels for the subset
#         example_weights - An array specifying the importance of each example
        
#     '''     

#     # make sure the batch size is a multiple of three
#     # to allow an equal number of positive and negative samples    
#     assert batch_size % 3 == 0
    
#     # Number of positive examples in each batch is half of the batch size
#     # same with number of negative examples in each batch
#     n_to_take = batch_size // 3
    
#     # Use pos_index to walk through the data_pos array
#     # same with neg_index and data_neg
#     pos_index = 0
#     neu_index
#     neg_index = 0
    
#     len_data_pos = len(data_pos)
#     len_data_neu = len(data_neu)
#     len_data_neg = len(data_neg)
    
#     # Get and array with the data indexes
#     pos_index_lines = list(range(len_data_pos))
#     neu_index_lines = list(range(len_data_neu))
#     neg_index_lines = list(range(len_data_neg))
    
#     # shuffle lines if shuffle is set to True
#     if shuffle:
#         rnd.shuffle(pos_index_lines)
#         rnd.shuffle(neu_index_lines)
#         rnd.shuffle(neg_index_lines)
        
#     stop = False
    
#     # Loop indefinitely
#     while not stop:  
        
#         # create a batch with positive, neutral and negative examples
#         batch = []
        
#         # First part: Pack n_to_take positive examples
        
#         # Start from 0 and increment i up to n_to_take
#         for i in range(n_to_take):
                    
#             # If the positive index goes past the positive dataset,
#             if pos_index >= len_data_pos: 
                
#                 # If loop is set to False, break once we reach the end of the dataset
#                 if not loop:
#                     stop = True;
#                     break;
#                 # If user wants to keep re-using the data, reset the index
#                 pos_index = 0
#                 if shuffle:
#                     # Shuffle the index of the positive sample
#                     rnd.shuffle(pos_index_lines)
                    
#             # get the tweet as pos_index
#             tweet = data_pos[pos_index_lines[pos_index]]
            
#             # convert the tweet into tensors of integers representing the processed words
#             tensor = tweet_to_tensor(tweet, vocab_dict)
            
#             # append the tensor to the batch list
#             batch.append(tensor)
            
#             # Increment pos_index by one
#             pos_index = pos_index + 1
            
#         for i in range(n_to_take):

#             if neu_index >= len_data_neu:
                
#                 if not loop:
#                     stop = True;
#                     break;
                    
#                 neu_index = 0
#                 if shuffle:
#                     rnd.shuffle(neu_index_lines)
                    
#             tweet = data_neu[neu_index_lines[neu_index]]
            
#         ### START CODE HERE (Replace instances of 'None' with your code) ###

#         # Second part: Pack n_to_take negative examples

#         # Using the same batch list, start from 0 and increment i up to n_to_take
#         for i in range(n_to_take):
            
#             # If the negative index goes past the negative dataset,
#             if neg_index >= len_data_neg:
                
#                 # If loop is set to False, break once we reach the end of the dataset
#                 if not loop:
#                     stop = True 
#                     break 
                    
#                 # If user wants to keep re-using the data, reset the index
#                 neg_index = 0
                
#                 if shuffle:
#                     # Shuffle the index of the negative sample
#                     rnd.shuffle(neg_index_lines)
                    
#             # get the tweet as neg_index
#             tweet = data_neg[neg_index_lines[neg_index]]
            
#             # convert the tweet into tensors of integers representing the processed words
#             tensor = tweet_to_tensor(tweet, vocab_dict)
            
#             # append the tensor to the batch list
#             batch.append(tensor)
            
#             # Increment neg_index by one
#             neg_index += 1

#         ### END CODE HERE ###        

#         if stop:
#             break;

#         # Get the max tweet length (the length of the longest tweet) 
#         # (you will pad all shorter tweets to have this length)
#         max_len = max([len(t) for t in batch]) 
        
        
#         # Initialize the input_l, which will 
#         # store the padded versions of the tensors
#         tensor_pad_l = []
#         # Pad shorter tweets with zeros
#         for tensor in batch:


#         ### START CODE HERE (Replace instances of 'None' with your code) ###
#             # Get the number of positions to pad for this tensor so that it will be max_len long
#             n_pad = max_len - len(tensor)
            
#             # Generate a list of zeros, with length n_pad
#             pad_l = [0]*n_pad
            
#             # concatenate the tensor and the list of padded zeros
#             tensor_pad = tensor + pad_l
            
#             # append the padded tensor to the list of padded tensors
#             tensor_pad_l.append(tensor_pad)

#         # convert the list of padded tensors to a numpy array
#         # and store this as the model inputs
#         inputs = np.array(tensor_pad_l)
  
#         # Generate the list of targets for the positive examples (a list of ones)
#         # The length is the number of positive examples in the batch
#         target_pos = [1]*n_to_take
        
#         # Generate the list of targets for the neutral examples (a list of zeros)
#         # The length is the number of positive examples in the batch
        
#         target_neu = [0]*n_to_take
        
#         # Generate the list of targets for the negative examples (a list of negative ones)
#         # The length is the number of negative examples in the batch
#         target_neg = [-1]*n_to_take
        
#         # Concatenate the positve and negative targets
#         target_l = target_pos + target_neu + target_neg
        
#         # Convert the target list into a numpy array
#         targets = np.array(target_l)

#         # Example weights: Treat all examples equally importantly.
#         example_weights = np.ones_like(targets)
        

#         ### END CODE HERE ###

#         # note we use yield and not return
#         yield inputs, targets, example_weights

In [131]:
def data_generator(data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):
    '''
    Input: 
        data_pos - Set of positive examples
        data_neu - Set of neutral examples
        data_neg - Set of negative examples
        batch_size - number of samples per batch. Must be a multiple of three
        loop - True or False
        vocab_dict - The words dictionary
        shuffle - Shuffle the data order
    Yield:
        inputs - Subset of positive and negative examples
        targets - The corresponding labels for the subset
        example_weights - An array specifying the importance of each example
        
    '''     

    # make sure the batch size is a multiple of three
    # to allow an equal number of positive and negative samples    
    assert batch_size % 2 == 0
    
    # Number of positive examples in each batch is half of the batch size
    # same with number of negative examples in each batch
    n_to_take = batch_size // 2
    
    # Use pos_index to walk through the data_pos array
    # same with neg_index and data_neg
    pos_index = 0
#     neu_index = 0
    neg_index = 0
    
    len_data_pos = len(data_pos)
#     len_data_neu = len(data_neu)
    len_data_neg = len(data_neg)
    
    # Get and array with the data indexes
    pos_index_lines = list(range(len_data_pos))
#     neu_index_lines = list(range(len_data_neu))
    neg_index_lines = list(range(len_data_neg))
    
    # shuffle lines if shuffle is set to True
    if shuffle:
        rnd.shuffle(pos_index_lines)
#         rnd.shuffle(neu_index_lines)
        rnd.shuffle(neg_index_lines)
        
    stop = False
    
    # Loop indefinitely
    while not stop:  
        
        # create a batch with positive, neutral and negative examples
        batch = []
        
        # First part: Pack n_to_take positive examples
        
        # Start from 0 and increment i up to n_to_take
        for i in range(n_to_take):
                    
            # If the positive index goes past the positive dataset,
            if pos_index >= len_data_pos: 
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                # If user wants to keep re-using the data, reset the index
                pos_index = 0
                if shuffle:
                    # Shuffle the index of the positive sample
                    rnd.shuffle(pos_index_lines)
                    
            # get the tweet as pos_index
            tweet = data_pos[pos_index_lines[pos_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment pos_index by one
            pos_index = pos_index + 1
            
#         for i in range(n_to_take):

#             if neu_index >= len_data_neu:
                
#                 if not loop:
#                     stop = True;
#                     break;
                    
#                 neu_index = 0
#                 if shuffle:
#                     rnd.shuffle(neu_index_lines)
                    
#             tweet = data_neu[neu_index_lines[neu_index]]
            
#             # convert the tweet into tensors of integers representing the processed words
#             tensor = tweet_to_tensor(tweet, vocab_dict)
            
#             # append the tensor to the batch list
#             batch.append(tensor)
            
#             # Increment pos_index by one
#             neu_index = neu_index + 1
            
        ### START CODE HERE (Replace instances of 'None' with your code) ###

        # Second part: Pack n_to_take negative examples

        # Using the same batch list, start from 0 and increment i up to n_to_take
        for i in range(n_to_take):
            
            # If the negative index goes past the negative dataset,
            if neg_index >= len_data_neg:
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True 
                    break 
                    
                # If user wants to keep re-using the data, reset the index
                neg_index = 0
                
                if shuffle:
                    # Shuffle the index of the negative sample
                    rnd.shuffle(neg_index_lines)
                    
            # get the tweet as neg_index
            tweet = data_neg[neg_index_lines[neg_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment neg_index by one
            neg_index += 1

        ### END CODE HERE ###        

        if stop:
            break;

        # Get the max tweet length (the length of the longest tweet) 
        # (you will pad all shorter tweets to have this length)
        max_len = max([len(t) for t in batch]) 
        
        
        # Initialize the input_l, which will 
        # store the padded versions of the tensors
        tensor_pad_l = []
        # Pad shorter tweets with zeros
        for tensor in batch:


        ### START CODE HERE (Replace instances of 'None' with your code) ###
            # Get the number of positions to pad for this tensor so that it will be max_len long
            n_pad = max_len - len(tensor)
            
            # Generate a list of zeros, with length n_pad
            pad_l = [0]*n_pad
            
            # concatenate the tensor and the list of padded zeros
            tensor_pad = tensor + pad_l
            
            # append the padded tensor to the list of padded tensors
            tensor_pad_l.append(tensor_pad)

        # convert the list of padded tensors to a numpy array
        # and store this as the model inputs
        inputs = np.array(tensor_pad_l)
  
        # Generate the list of targets for the positive examples (a list of ones)
        # The length is the number of positive examples in the batch
        target_pos = [1]*n_to_take
        
        # Generate the list of targets for the neutral examples (a list of zeros)
        # The length is the number of positive examples in the batch
        
        target_neg = [0]*n_to_take
        
        # Generate the list of targets for the negative examples (a list of negative ones)
        # The length is the number of negative examples in the batch
#         target_neg = [-1]*n_to_take
        
        # Concatenate the positve and negative targets
#         target_l = target_pos + target_neu + target_neg
        target_l = target_pos + target_neg
        
        # Convert the target list into a numpy array
        targets = np.array(target_l)

        # Example weights: Treat all examples equally importantly.
        example_weights = np.ones_like(targets)
        

        ### END CODE HERE ###

        # note we use yield and not return
        yield inputs, targets, example_weights

In [119]:
# # Set the random number generator for the shuffle procedure
# rnd.seed(30) 

# # Create the training data generator

# def train_generator(batch_size, train_pos, train_neu, train_neg, vocab_dict, loop=True, shuffle = False):
#     return data_generator(train_pos, train_neu, train_neg, batch_size, loop, vocab_dict, shuffle)

# # Create the testidation data generator
# def test_generator(batch_size, test_pos, test_neu
#                     , test_neg, vocab_dict, loop=True
#                     , shuffle = False):
#     return data_generator(test_pos, test_neu, test_neg, batch_size, loop, vocab_dict, shuffle)

# # Create the testidation data generator
# def test_generator(batch_size, test_pos, test_neu
#                     , test_neg, vocab_dict, loop=False
#                     , shuffle = False):
#     return data_generator(test_pos, test_neu, test_neg, batch_size, loop, vocab_dict, shuffle)

# # Get a batch from the train_generator and inspect.
# inputs, targets, example_weights = next(train_generator(6, train_pos, train_neu, train_neg, Vocab, shuffle=True))

# # this will print a list of 4 tensors padded with zeros
# print(f'Inputs: {inputs}')
# print(f'Targets: {targets}')
# print(f'Example Weights: {example_weights}')

In [132]:
# Set the random number generator for the shuffle procedure
rnd.seed(30) 

# Create the training data generator

def train_generator(batch_size, train_pos, train_neg, vocab_dict, loop=True, shuffle = False):
    return data_generator(train_pos, train_neg, batch_size, loop, vocab_dict, shuffle)

# Create the testidation data generator
def test_generator(batch_size, test_pos
                    , test_neg, vocab_dict, loop=True
                    , shuffle = False):
    return data_generator(test_pos, test_neg, batch_size, loop, vocab_dict, shuffle)

# Create the testidation data generator
def test_generator(batch_size, test_pos
                    , test_neg, vocab_dict, loop=False
                    , shuffle = False):
    return data_generator(test_pos, test_neg, batch_size, loop, vocab_dict, shuffle)

# Get a batch from the train_generator and inspect.
inputs, targets, example_weights = next(train_generator(4, train_pos, train_neg, Vocab, shuffle=True))

# this will print a list of 4 tensors padded with zeros
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')

Inputs: [[ 266  260  267  647  270 2224 3720 1587  266  260 3248  502 1913  827
   703  125 3721  895  238 1911   68  263 3722  234  404  392   35  127
  3723]
 [ 383  111 1262  168  266 1264 1265 1264  383   54   17   19   20    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [  90 6236   90 2066 6236 5971 1457 5677    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [  29  646 6458 3638 6285 5845 6458 4590 1703 6459 6254 1211   17   19
    20    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]]
Targets: [1 1 0 0]
Example Weights: [1 1 1 1]


In [134]:
inputs, targets, example_weights = next(train_generator(6, train_pos, train_neg, Vocab, shuffle=True))

# this will print a list of 4 tensors padded with zeros
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')

Inputs: [[1187 1188 1189 1190   13   31   81  142 1191 1192 1193 1194 1195    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  31   81   13 3209 1228 1720  798  472 3210   81 1409    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [ 332  333  334  335  336    7  290    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [4367 7012  745  572  200   17  422 7013  190 1515   17  422 7013  190
  1515 7014 3256 1556 2365  524 6502 6675   17   19   20]
 [1100   62 1327 2057   47  221  628 6338  497 2736   17   19   20    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  29  299 1652  517 6479 1385 6480    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Targets: [1 1 1 0 0 0]
Example Weights: [1 1 1 1 1 1]


In [136]:
# Test the train_generator

# Create a data generator for training data,
# which produces batches of size 4 (for tensors and their respective targets)
tmp_data_gen = train_generator(batch_size = 6, train_pos=train_pos, train_neg=train_neg, vocab_dict=Vocab)

# Call the data generator to get one batch and its targets
tmp_inputs, tmp_targets, tmp_example_weights = next(tmp_data_gen)

print(f"The inputs shape is {tmp_inputs.shape}")
for i,t in enumerate(tmp_inputs):
    print(f"input tensor: {t}; target {tmp_targets[i]}; example weights {tmp_example_weights[i]}")

The inputs shape is (6, 21)
input tensor: [ 3  4  5  6  7  8  7  9 10 11 12 13 14 13 15 16 17 18 17 19 20]; target 1; example weights 1
input tensor: [21 22 23 24 25 26 27 28 29 30 31 32 33 25 22 34 35 36  0  0  0]; target 1; example weights 1
input tensor: [37 29 38 39 40 35 41 42 43  0  0  0  0  0  0  0  0  0  0  0  0]; target 1; example weights 1
input tensor: [  17  251 5430 1389  238  642  118    0    0    0    0    0    0    0
    0    0    0    0    0    0    0]; target 0; example weights 1
input tensor: [ 820  342  171 4305   99 1545  412   31    0    0    0    0    0    0
    0    0    0    0    0    0    0]; target 0; example weights 1
input tensor: [  90 6236   90 2066 6236 5971 1457 5677    0    0    0    0    0    0
    0    0    0    0    0    0    0]; target 0; example weights 1


### Defining the layer class

In [137]:
class Layer(object):
    """ Base class for layers.
    """

    # Constructor
    def __init__(self):
        # set weights to None
        self.weights = None

    # The forward propagation should be implemented
    # by subclasses of this Layer class
    def forward(self, x):
        raise NotImplementedError

    # This function initializes the weights
    # based on the input signature and random key,
    # should be implemented by subclasses of this Layer class
    def init_weights_and_state(self, input_signature, random_key):
        pass

    # This initializes and returns the weights, do not override.
    def init(self, input_signature, random_key):
        self.init_weights_and_state(input_signature, random_key)
        return self.weights

    # __call__ allows an object of this class
    # to be called like it's a function.
    def __call__(self, x):
        # When this layer object is called, 
        # it calls its forward propagation function
        return self.forward(x)

In [138]:
class Relu(Layer):
    """Relu activation function implementation"""
    def forward(self, x):
        '''
        Input: 
            - x (a numpy array): the input
        Output:
            - activation (numpy array): all positive or 0 version of x
        '''
        activation = np.maximum(x,0)
        
        return activation

In [139]:
class Dense(Layer):
    """
    A dense (fully-connected) layer.
    """

    # __init__ is implemented for you
    def __init__(self, n_units, init_stdev=0.1):
        
        # Set the number of units in this layer
        self._n_units = n_units
        self._init_stdev = init_stdev

    # Please implement 'forward()'
    def forward(self, x):

        # Matrix multiply x and the weight matrix
        dense = np.dot(x, self.weights)

        return dense

    # init_weights
    def init_weights_and_state(self, input_signature, random_key):
        
        input_shape = input_signature.shape
       
        w = self._init_stdev * trax.fastmath.random.normal(key = random_key, shape = (input_shape[-1], self._n_units))
 
        self.weights = w
        return self.weights

In [141]:
def classifier(vocab_size=9914, embedding_dim=256, output_dim=2, mode='train'):

    # create embedding layer
    embed_layer = tl.Embedding( 
        vocab_size=vocab_size, # Size of the vocabulary
        d_feature=embedding_dim # Embedding dimension
    ) 
    
    # Create a mean layer, to create an "average" word embedding
    mean_layer = tl.Mean(axis=1)
    
    # Create a dense layer, one unit for each output
    dense_output_layer = tl.Dense(n_units = output_dim)
    
    # Create the log softmax layer (no parameters needed)
    log_softmax_layer = tl.LogSoftmax()
    
    # Use tl.Serial to combine all layers
    # and create the classifier
    # of type trax.layers.combinators.Serial
    model = tl.Serial( 
      embed_layer, # embedding layer
      mean_layer, # mean layer
      dense_output_layer, # dense output layer
      log_softmax_layer, # log softmax layer
    ) 
    ### END CODE HERE ###
    
    # return the model of type
    return model

In [146]:
from trax.supervised import training

def get_train_eval_tasks(train_pos, train_neg, test_pos, test_neg, vocab_dict, loop, batch_size = 16):
    
    rnd.seed(271)

    train_task = training.TrainTask(
        labeled_data=train_generator(batch_size, train_pos
                    , train_neg, vocab_dict, loop
                    , shuffle = True),
        loss_layer=tl.WeightedCategoryCrossEntropy(),
        optimizer=trax.optimizers.Adam(0.01),
        n_steps_per_checkpoint=10,
    )

    eval_task = training.EvalTask(
        labeled_data=test_generator(batch_size, test_pos
                    , test_neg, vocab_dict, loop
                    , shuffle = True),        
        metrics=[tl.WeightedCategoryCrossEntropy(), tl.WeightedCategoryAccuracy()],
    )
    
    return train_task, eval_task
    

train_task, eval_task = get_train_eval_tasks(train_pos, train_neg, test_pos, test_neg, Vocab, True, batch_size = 24)
model = classifier()

In [147]:
model

Serial[
  Embedding_9914_256
  Mean
  Dense_2
  LogSoftmax
]

In [157]:
dir_path = './model/'

try:
    shutil.rmtree(dir_path)
except OSError as e:
    pass


output_dir = './model/'
output_dir_expand = os.path.expanduser(output_dir)
print(output_dir_expand)

./model/


In [158]:
# UNQ_C6 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: train_model
def train_model(classifier, train_task, eval_task, n_steps, output_dir):
    '''
    Input: 
        classifier - the model you are building
        train_task - Training task
        eval_task - Evaluation task. Received as a list.
        n_steps - the evaluation steps
        output_dir - folder to save your files
    Output:
        trainer -  trax trainer
    '''
    rnd.seed(31) # Do NOT modify this random seed. This makes the notebook easier to replicate
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###          
    training_loop = training.Loop( 
                                classifier, # The learning model
                                train_task, # The training task
                                eval_tasks=eval_task, # The evaluation task
                                output_dir=output_dir, # The output directory
                                random_seed=31 # Do not modify this random seed in order to ensure reproducibility and for grading purposes.
    ) 

    training_loop.run(n_steps = n_steps)
    ### END CODE HERE ###
    
    # Return the training_loop, since it has the model.
    return training_loop

In [159]:
training_loop = train_model(model, train_task, [eval_task], 100, output_dir_expand)


Step      1: Total number of trainable weights: 2538498
Step      1: Ran 1 train steps in 1.18 secs
Step      1: train WeightedCategoryCrossEntropy |  0.33064303
Step      1: eval  WeightedCategoryCrossEntropy |  0.46775231
Step      1: eval      WeightedCategoryAccuracy |  0.79166669

Step     10: Ran 9 train steps in 3.29 secs
Step     10: train WeightedCategoryCrossEntropy |  0.43946570
Step     10: eval  WeightedCategoryCrossEntropy |  0.50949985
Step     10: eval      WeightedCategoryAccuracy |  0.83333331

Step     20: Ran 10 train steps in 3.21 secs
Step     20: train WeightedCategoryCrossEntropy |  0.42646426
Step     20: eval  WeightedCategoryCrossEntropy |  0.38794735
Step     20: eval      WeightedCategoryAccuracy |  0.75000000

Step     30: Ran 10 train steps in 3.72 secs
Step     30: train WeightedCategoryCrossEntropy |  0.39143488
Step     30: eval  WeightedCategoryCrossEntropy |  0.55485648
Step     30: eval      WeightedCategoryAccuracy |  0.79166669

Step     40: Ran 

In [163]:
# Create a generator object
tmp_train_generator = train_generator(16, train_pos
                    , train_neg, Vocab, loop=True
                    , shuffle = False)



# get one batch
tmp_batch = next(tmp_train_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

print(f"The batch is a tuple of length {len(tmp_batch)} because position 0 contains the tweets, and position 1 contains the targets.") 
print(f"The shape of the tweet tensors is {tmp_inputs.shape} (num of examples, length of tweet tensors)")
print(f"The shape of the labels is {tmp_targets.shape}, which is the batch size.")
print(f"The shape of the example_weights is {tmp_example_weights.shape}, which is the same as inputs/targets size.")

The batch is a tuple of length 3 because position 0 contains the tweets, and position 1 contains the targets.
The shape of the tweet tensors is (16, 28) (num of examples, length of tweet tensors)
The shape of the labels is (16,), which is the batch size.
The shape of the example_weights is (16,), which is the same as inputs/targets size.


In [160]:
def compute_accuracy(preds, y, y_weights):
    """
    Input: 
        preds: a tensor of shape (dim_batch, output_dim) 
        y: a tensor of shape (dim_batch,) with the true labels
        y_weights: a n.ndarray with the a weight for each example
    Output: 
        accuracy: a float between 0-1 
        weighted_num_correct (np.float32): Sum of the weighted correct predictions
        sum_weights (np.float32): Sum of the weights
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # Create an array of booleans, 
    # True if the probability of positive sentiment is greater than
    # the probability of negative sentiment
    # else False
    is_pos = preds[:, 1] > preds[:, 0] 

    # convert the array of booleans into an array of np.int32
    is_pos_int = is_pos.astype(np.int32)
    
    # compare the array of predictions (as int32) with the target (labels) of type int32
    correct = is_pos_int == y

    # Count the sum of the weights.
    sum_weights = np.sum(y_weights)
    
    # convert the array of correct predictions (boolean) into an arrayof np.float32
    correct_float = correct.astype(np.float32)
    
    # Multiply each prediction with its corresponding weight.
    weighted_correct_float = correct_float * y_weights

    # Sum up the weighted correct predictions (of type np.float32), to go in the
    # denominator.
    weighted_num_correct = np.sum(weighted_correct_float)

    # Divide the number of weighted correct predictions by the sum of the
    # weights.
    accuracy = weighted_num_correct / sum_weights

    ### END CODE HERE ###
    return accuracy, weighted_num_correct, sum_weights

In [164]:
# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)
print(f"The prediction shape is {tmp_pred.shape}, num of tensor_tweets as rows")
print("Column 0 is the probability of a negative sentiment (class 0)")
print("Column 1 is the probability of a positive sentiment (class 1)")
print()
print("View the prediction array")
tmp_pred

The prediction shape is (16, 2), num of tensor_tweets as rows
Column 0 is the probability of a negative sentiment (class 0)
Column 1 is the probability of a positive sentiment (class 1)

View the prediction array


DeviceArray([[-7.7691083e+00, -4.2271614e-04],
             [-5.5346572e-01, -8.5555720e-01],
             [-1.9592929e+00, -1.5193754e-01],
             [-2.1556168e+00, -1.2310791e-01],
             [-2.0442739e+00, -1.3865787e-01],
             [-3.7320912e-01, -1.1664240e+00],
             [-4.8675847e+00, -7.7216625e-03],
             [-7.9531021e+00, -3.5166740e-04],
             [-3.7326813e-03, -5.5924702e+00],
             [-3.1380653e-03, -5.7657194e+00],
             [-7.5914860e-03, -4.8845310e+00],
             [-3.7376881e-03, -5.5911465e+00],
             [-4.8426688e-01, -9.5750010e-01],
             [-1.9037724e-03, -6.2647991e+00],
             [-1.0868633e-01, -2.2731400e+00],
             [-1.1870861e-03, -6.7368240e+00]], dtype=float32)

In [165]:
# turn probabilites into category predictions
tmp_is_positive = tmp_pred[:,1] > tmp_pred[:,0]
for i, p in enumerate(tmp_is_positive):
    print(f"Neg log prob {tmp_pred[i,0]:.4f}\tPos log prob {tmp_pred[i,1]:.4f}\t is positive? {p}\t actual {tmp_targets[i]}")

Neg log prob -7.7691	Pos log prob -0.0004	 is positive? True	 actual 1
Neg log prob -0.5535	Pos log prob -0.8556	 is positive? False	 actual 1
Neg log prob -1.9593	Pos log prob -0.1519	 is positive? True	 actual 1
Neg log prob -2.1556	Pos log prob -0.1231	 is positive? True	 actual 1
Neg log prob -2.0443	Pos log prob -0.1387	 is positive? True	 actual 1
Neg log prob -0.3732	Pos log prob -1.1664	 is positive? False	 actual 1
Neg log prob -4.8676	Pos log prob -0.0077	 is positive? True	 actual 1
Neg log prob -7.9531	Pos log prob -0.0004	 is positive? True	 actual 1
Neg log prob -0.0037	Pos log prob -5.5925	 is positive? False	 actual 0
Neg log prob -0.0031	Pos log prob -5.7657	 is positive? False	 actual 0
Neg log prob -0.0076	Pos log prob -4.8845	 is positive? False	 actual 0
Neg log prob -0.0037	Pos log prob -5.5911	 is positive? False	 actual 0
Neg log prob -0.4843	Pos log prob -0.9575	 is positive? False	 actual 0
Neg log prob -0.0019	Pos log prob -6.2648	 is positive? False	 actual 

In [166]:
tmp_val_generator = train_generator(64, train_pos
                    , train_neg, Vocab, loop=True
                    , shuffle = False)

# get one batch
tmp_batch = next(tmp_train_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)
tmp_acc, tmp_num_correct, tmp_num_predictions = compute_accuracy(preds=tmp_pred, y=tmp_targets, y_weights=tmp_example_weights)

print(f"Model's prediction accuracy on a single training batch is: {100 * tmp_acc}%")
print(f"Weighted number of correct predictions {tmp_num_correct}; weighted number of total observations predicted {tmp_num_predictions}")

Model's prediction accuracy on a single training batch is: 100.0%
Weighted number of correct predictions 16.0; weighted number of total observations predicted 16


In [167]:
def test_model(generator, model, compute_accuracy=compute_accuracy):
    '''
    Input: 
        generator: an iterator instance that provides batches of inputs and targets
        model: a model instance 
    Output: 
        accuracy: float corresponding to the accuracy
    '''
    
    accuracy = 0.
    total_num_correct = 0
    total_num_pred = 0
        
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    for batch in generator: 
        
        # Retrieve the inputs from the batch
        inputs = batch[0]
        
        # Retrieve the targets (actual labels) from the batch
        targets = batch[1]
        
        # Retrieve the example weight.
        example_weight = batch[2]

        # Make predictions using the inputs            
        pred = model(inputs)
        
        # Calculate accuracy for the batch by comparing its predictions and targets
        batch_accuracy, batch_num_correct, batch_num_pred = compute_accuracy(pred, targets, example_weight)
                
        # Update the total number of correct predictions
        # by adding the number of correct predictions from this batch
        total_num_correct += batch_num_correct
        
        # Update the total number of predictions 
        # by adding the number of predictions made for the batch
        total_num_pred += batch_num_pred

    # Calculate accuracy over all examples
    accuracy = total_num_correct / total_num_pred
    
    ### END CODE HERE ###
    return accuracy

In [169]:
model = training_loop.eval_model
accuracy = test_model(test_generator(16, test_pos
                    , test_neg, Vocab, loop=False
                    , shuffle = False), model)

print(f'The accuracy of your model on the validation set is {accuracy:.4f}', )

The accuracy of your model on the validation set is 0.8153
